<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Boston 데이터를 대상으로 모델링를 진행합니다.
- 개별 모델과 스태킹(Stacking) 모델의 관계를 확인합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [4]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [5]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/diabetes.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [7]:
# 상위 몇 개 행 확인
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
# 하위 몇 개 행 확인
data.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [9]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [12]:
# target 확인
target = 'Outcome'

# 데이터 분리
x = data.drop(columns=target)
y = data.loc[:, target]

**2) 학습용, 평가용 데이터 분리**

In [14]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) 

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 우선 회귀 문제인지 분류 문제인지 명확히 구분합니다.

**1) 불러오기**

- 사용할 라이브러리를 모두 불러옵니다.

In [17]:
# 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
from lightgbm import LGBMClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import *

**2) 개별 모델 선언과 학습**

- 개별 모델을 선언하고 학습합니다.

In [19]:
# KNN
model_knn = make_pipeline(MinMaxScaler(), KNeighborsClassifier())
model_knn.fit(x_train, y_train)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=1)
model_dt.fit(x_train, y_train)

# Logistic Regression
model_lr = LogisticRegression()
model_lr.fit(x_train, y_train)

# Light GBM
model_lgb = LGBMClassifier(random_state=1, verbose=-1)
model_lgb.fit(x_train, y_train)

LGBMClassifier(random_state=1, verbose=-1)

**3) 최종 모델 만들기**

- 개별 모델의 예측값으로 학습한 최종 모델을 만듭니다.

In [21]:
# 예측 결과 수집 (교차검증으로 훈련 , 모델 생성)
pred_dict = {'p1': cross_val_predict(model_lr, x_train, y_train, cv=5),
             'p2': cross_val_predict(model_knn, x_train, y_train, cv=5),
             'p3': cross_val_predict(model_dt, x_train, y_train, cv=5),
             'p4': cross_val_predict(model_lgb, x_train, y_train, cv=5)}

# 데이터프레임 선언
result = pd.DataFrame(pred_dict)
result

,p1,p2,p3,p4
0,1,1,1,0
1,0,0,0,1
2,0,0,0,0
3,0,1,1,1
4,0,0,0,0
...,...,...,...,...
532,1,1,0,1
533,1,1,1,1
534,1,0,0,0
535,1,1,1,1


In [22]:
# 최종 모델 학습 렌덤 포레스트 생성
final_model = RandomForestClassifier(random_state=1)
final_model.fit(result, y_train)

RandomForestClassifier(random_state=1)

**4) 예측하고 평가**

- 개별 모델이 예측한 값으로 최종 모델이 학습해 다시 예측합니다.

In [24]:
# 예측 결과 수집 (검증용 데이터로 실제 예측값 확인 )
pred_dict = {'p1': model_lr.fit(x_train, y_train).predict(x_test),
             'p2': model_knn.fit(x_train, y_train).predict(x_test),
             'p3': model_dt.fit(x_train, y_train).predict(x_test),
             'p4': model_lgb.fit(x_train, y_train).predict(x_test)}

# 데이터프레임 선언
result = pd.DataFrame(pred_dict)

# 최종 모델 예측
y_pred = final_model.predict(result)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[131  15]
 [ 34  51]]
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       146
           1       0.77      0.60      0.68        85

    accuracy                           0.79       231
   macro avg       0.78      0.75      0.76       231
weighted avg       0.79      0.79      0.78       231



**5) 스태킹 보델 사용**

- 스캐팅 모델을 만들고 학습한 후 예측해 봅니다. 

In [26]:
# 스태킹 모델 선언
estimators = [('lr', model_lr), 
              ('dt', model_dt), 
              ('knn', model_knn), 
              ('lgb', model_lgb)]

model = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(random_state=1))

# 학습하기
model.fit(x_train, y_train)

# 예측하기
y_pred = model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[121  25]
 [ 32  53]]
              precision    recall  f1-score   support

           0       0.79      0.83      0.81       146
           1       0.68      0.62      0.65        85

    accuracy                           0.75       231
   macro avg       0.74      0.73      0.73       231
weighted avg       0.75      0.75      0.75       231



- 개별 모델 성능을 확인합니다.

In [28]:
# 개별 모델 성능 확인
for m in [model_knn, model_dt, model_lr, model_lgb]:
    m.fit(x_train, y_train)
    y_pred = m.predict(x_test)
    print(f'* {m.__class__.__name__} Accuracy Score: {accuracy_score(y_test, y_pred):.4f}')

* Pipeline Accuracy Score: 0.7749
* DecisionTreeClassifier Accuracy Score: 0.6883
* LogisticRegression Accuracy Score: 0.7835
* LGBMClassifier Accuracy Score: 0.7662


# 5.성능 튜닝

- GridSearchCV로 개별 모델의 성능을 최적화합니다.
- 성능이 최적화된 개별 모델을 사용하는 보팅 모델을 만들고 성능을 확인합니다.

**1) 개별 모델 성능 최적화**

- GridSearchCV로 개별 모델의 성능을 최적화합니다.

In [31]:
# KNN
param = {'kneighborsclassifier__n_neighbors': range(2, 11, 1)}
model_grid = GridSearchCV(make_pipeline(MinMaxScaler(), KNeighborsClassifier()), param)
model_grid.fit(x_train, y_train)
model_knn = model_grid.best_estimator_

# Decision Tree
param = {'max_depth': range(1, 11, 1)}
model_grid = GridSearchCV(DecisionTreeClassifier(), param)
model_grid.fit(x_train, y_train)
model_dt = model_grid.best_estimator_

# Logistic Regression
model_lr = LogisticRegression()
model_lr = model_lr.fit(x_train, y_train)

# LightGBM
param = {'max_depth': range(1, 11, 1)}
model_grid = GridSearchCV(LGBMClassifier(verbose=-1), param)
model_grid.fit(x_train, y_train)
model_lgb = model_grid.best_estimator_

**2) 스태킹 모델 선언**

- 성능이 최적화된 개별 모델을 사용하는 스태킹 모델을 만들고 성능을 확인합니다.

In [33]:
# 스태킹 모델 선언
estimators = [('lr', model_lr), 
              ('dt', model_dt), 
              ('knn', model_knn), 
              ('lgb', model_lgb)]

model = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier(random_state=1))

# 학습하기
model.fit(x_train, y_train)

# 예측하기
y_pred = model.predict(x_test)

# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[123  23]
 [ 27  58]]
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       146
           1       0.72      0.68      0.70        85

    accuracy                           0.78       231
   macro avg       0.77      0.76      0.76       231
weighted avg       0.78      0.78      0.78       231



- 개별 모델 성능도 확인합니다.

In [35]:
# 개별 모델 성능 확인
for m in [model_knn, model_dt, model_lr, model_lgb]:
    y_pred = m.predict(x_test)
    print(f'* {m.__class__.__name__} Accuracy Score: {accuracy_score(y_test, y_pred):.4f}')

* Pipeline Accuracy Score: 0.7835
* DecisionTreeClassifier Accuracy Score: 0.7706
* LogisticRegression Accuracy Score: 0.7835
* LGBMClassifier Accuracy Score: 0.7879
